In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mxnet 
from mxnet.gluon import *


In [ ]:
finalData = pd.read_csv('../data/ReadyData.csv')


In [ ]:
finalData.info()

In [ ]:
finalData.head()

In [ ]:
lst = ["datetime"] + list(finalData.columns)[1:]
lst

In [ ]:
finalData.columns = lst

In [ ]:
finalData.head()

In [ ]:
len(finalData[finalData.year == 2017].month.unique()) #no of months in 2017
len(finalData[finalData.year == 2018].month.unique()) #no of months in 2018

In [ ]:
finalData[finalData.year == 2017].info() #no of values in 2017
finalData[finalData.year == 2018].info() #no of values in 2018

In [ ]:
#we have data for a total of 20 months, let's train (roughly) for 16 months, and test for the remaining 4.

trainData = finalData[:30*16*24]
testData = finalData[30*16*24:]
trainData.info()
testData.info()
finalData.tail()

In [ ]:
"""
Train = trainData.shape
tau = 5
features = np.zeros((Train[0]-tau, tau*Train[1]-1))
features = pd.DataFrame(features)
print(features.shape)
for i in range(tau):
    features[:, i*Train[1]:(i+1)*Train[1]] = trainData[i:Train[0]-tau]
print(features)
"""

In [ ]:
finalData.drop("datetime", axis = 1, inplace = True)

In [ ]:


#finalData.drop("PM2.5", axis = 1, inplace = True)

In [ ]:
mean = finalData.mean()
min = finalData.min()
max = finalData.max()
finalData = finalData.apply(lambda x: (x - np.mean(x)) / (np.std(x)))

In [ ]:
name = list(finalData.columns)
cols, names = list(), list()
for i in range(7, 0, -1): #24 times steps
    cols.append(finalData.shift(i))
    names += [('%s(t-%d)' % (name[j], i)) for j in range(finalData.shape[1])]
#cols.append(finalData.shift(-i))
#names += [('%s(t)' % (name[j])) for j in range(finalData.shape[1])]
finalDataset = pd.concat(cols, axis=1)
finalDataset.columns = names
finalDataset = finalDataset[7:]
labels = finalData["PM2.5"]
display(finalDataset.head())
#labels = finalDataset["PM2.5(t)"]
#finalDataset.drop("PM2.5(t)", axis = 1, inplace = True)
#display(finalDataset)

In [ ]:
#finalDataset = finalDataset[:8000]
finalDataset.reset_index(drop = True)
finalDataset = finalDataset[:-1]
labels = labels[:-1]
#we have data for a total of 20 months, let's train (roughly) for 16 months, and test for the remaining 4.
trainDataX = finalDataset[:30*16*24]
testDataX = finalDataset[30*16*24:]
trainDataY = labels[:30*16*24]
testDataY = labels[30*16*24:]

#trainData.info()
#testData.info()


In [ ]:
finalDataset.info()

In [ ]:
from mxnet import nd
finalDataset = nd.array(finalDataset).reshape(-1, 7, 13)
trainDataX = nd.array(trainDataX).reshape(-1, 7, 13)
testDataX = nd.array(testDataX).reshape(-1, 7, 13)

In [ ]:
trainDataY = nd.array(trainDataY)
testDataY = nd.array(testDataY)

In [ ]:
model = mxnet.gluon.nn.Sequential()
model.add(mxnet.gluon.rnn.LSTM(13, 1, dropout = 0.2))
#model.add(mxnet.gluon.rnn.LSTM(30, 1, dropout = 0.2))
#model.add(mxnet.gluon.rnn.LSTM(10, 1, dropout = 0.2))
model.add(mxnet.gluon.nn.Dense(1, activation='tanh'))

model.initialize(mxnet.init.Xavier())

trainer = mxnet.gluon.Trainer(
    params=model.collect_params(),
    optimizer='adam',
    optimizer_params={'learning_rate': 0.05}
)

In [ ]:
loss = mxnet.gluon.loss.L2Loss()
error = mxnet.metric.MSE()

In [ ]:
epochs = 10
trainingLoss = []
train_iter = mxnet.io.NDArrayIter(trainDataX, trainDataY, 12, shuffle=True)
for epoch in range(epochs):
    for trn_batch in train_iter:
        x = trn_batch.data[0]
        y = trn_batch.label[0]
        
        with mxnet.autograd.record():
            y_pred = model(x)
            im_loss = loss(y_pred, y)
        
        #backprop
        im_loss.backward()
        
        #Optimize!
        trainer.step(batch_size=12)
        
    train_iter.reset()
    
    # Calculate train metrics
    
    predictions = model(trainDataX)
    error.update(trainDataY, predictions)
    trainingLoss.append(error.get()[1])
    error.reset()
    
    train_iter.reset()
    
    print("epoch: {} | trn_loss: {:.8f}".format(epoch+1,
                                                trainingLoss[-1]))


In [ ]:
print(y_pred)

In [ ]:
predictions = model(testDataX)
plt.plot(testDataY[:100].asnumpy(), label = 'actual')
plt.plot(predictions[:100].asnumpy())
plt.legend()
